In [57]:
from bs4 import BeautifulSoup
import requests

#result  = requests.get("https://en.wikipedia.org/wiki/W16_engine")
#result  = requests.get("https://en.wikipedia.org/wiki/Internal_combustion_engine")
result  = requests.get("https://en.wikipedia.org/wiki/Gas_turbine")

content = result.content
soup = BeautifulSoup(content)

/home/irza/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/irza/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [58]:
#delete script tags
for x in soup.findAll('script'):
    x.extract()

In [59]:
#get all paragraph element

p_el = soup.find_all('p')

all_text=""

for x in p_el:    
    all_text=all_text+x.get_text()

all_text

'An internal combustion engine (ICE) is a heat engine where the combustion of a fuel occurs with an oxidizer (usually air) in a combustion chamber that is an integral part of the working fluid flow circuit. In an internal combustion engine, the expansion of the high-temperature and high-pressure gases produced by combustion applies direct force to some component of the engine. The force is applied typically to pistons, turbine blades, rotor or a nozzle. This force moves the component over a distance, transforming chemical energy into useful mechanical energy.The first commercially successful internal combustion engine was created by Étienne Lenoir around 1859[1] and the first modern internal combustion engine was created in 1876 by Nikolaus Otto (see Otto engine).The term internal combustion engine usually refers to an engine in which combustion is intermittent, such as the more familiar four-stroke and two-stroke piston engines, along with variants, such as the six-stroke piston engin

In [60]:
#tokenize text
import nltk
import pandas as pd
import numpy as np

token = nltk.word_tokenize(all_text)
len(token)

9865

In [61]:
prv = nltk.word_tokenize(all_text)
prv.pop()
prv.insert(0,'')
len(prv)

9865

In [62]:
nxt = nltk.word_tokenize(all_text)
nxt.pop(0)
nxt.append('')
len(nxt)

9865

In [63]:
#find all entities 
import re

entities=None

for x in p_el:
    a_el=x.find_all('a', {'href': re.compile(r'^/wiki/')})
    
    if len(a_el) > 0:
        if entities is None:
            entities=a_el
        else:            
            entities=entities+a_el

len(entities)

391

In [64]:
#get previous and next token of entity
import bs4

ent_prev=[]
ent_next=[]
ent=[]

def get_token_from_sibling(el,src):    
    res=''
    
    if el is None:                
        if src=="prev":
            res=''
        elif src=="next":
            res=''
            
    elif isinstance(el, bs4.element.Tag):
        txt=el.get_text()
        tkn=nltk.word_tokenize(txt)

        if len(tkn) == 0:            
            if src=="prev":
                #print(t,src,type(el.previous_sibling))
                res=get_token_from_sibling(el.previous_sibling,src)
            elif src=="next":
                #print(t,src,type(el.next_sibling))
                res=get_token_from_sibling(el.next_sibling,src)
                
        elif src=="prev":
            res=tkn.pop()
        elif src=="next":
            res=tkn.pop(0)        
            
    elif isinstance(el, bs4.element.NavigableString):        
        tkn=nltk.word_tokenize(el)
        
        if len(tkn) == 0:               
            if src=="prev":
                #print(t,src,type(el.previous_sibling))
                res=get_token_from_sibling(el.previous_sibling,src)
            elif src=="next":
                #print(t,src,type(el.next_sibling))
                res=get_token_from_sibling(el.next_sibling,src)
                
        elif src=="prev":
            res=tkn.pop()
        elif src=="next":
            res=tkn.pop(0)
    
    return res

for x in entities:    
    ent_tkn=nltk.word_tokenize(x.get_text())
    
    for idx, t in enumerate(ent_tkn):    
        ent.append(t)
        
        if ((idx+1)==len(ent_tkn)) & (idx==0): 
            ent_prev.append(get_token_from_sibling(x.previous_sibling,"prev"))
            ent_next.append(get_token_from_sibling(x.next_sibling,"next"))
        
        elif idx==0: 
            ent_prev.append(get_token_from_sibling(x.previous_sibling,"prev"))
            ent_next.append(ent_tkn[idx+1])
                
        elif (idx+1)==len(ent_tkn): 
            ent_prev.append(ent_tkn[idx-1])
            ent_next.append(get_token_from_sibling(x.next_sibling,"next"))
        
        else:
            ent_prev.append(ent_tkn[idx-1])
            ent_next.append(ent_tkn[idx+1])
                            

In [65]:
print(len(ent_prev))
print(len(ent_next))
print(len(ent))
len(entities)

705
705
705


391

In [66]:
os.path.exists("./result/output-1.txt")

True

In [67]:
#save as text file

import os.path

directory="./result/"
num=0
file_path=""
file_exist=True

while file_exist:
    num=num+1
    file_path=directory+"output-"+str(num)+".txt"
    file_exist=os.path.exists(file_path)

print(file_path)
    
with open(file_path, "w") as text_file:    
    for idxt, t in enumerate(token):
        label="O"
        eq_ent=""
        eq_ent_prev=""
        eq_ent_next=""
        connector=""
        
        for idxet, et in enumerate(ent):
            if t == et:
                if (prv[idxt]==ent_prev[idxet]) & (nxt[idxt]==ent_next[idxet]):
                    label="I"
                    eq_ent=et
                    eq_ent_prev=ent_prev[idxet]
                    eq_ent_next=ent_next[idxet]
                    connector=' === '
        
        #text_file.write(t + ' ' + label + connector + eq_ent_prev + " " + eq_ent + " " + eq_ent_next + " " '\n')
        text_file.write(t + ' ' + label + '\n')
        
print("done")

./result/output-2.txt
done
